In [1]:
import pandas as pd
import mosek
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from copy import deepcopy
from torch import optim
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import cvxpy as cp
from torch.utils.data.sampler import SubsetRandomSampler


seed = 2023
torch.manual_seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(True, warn_only=True)
erm_ = False

data_train = pd.read_csv('adult.data', header=None)
data_test = pd.read_csv('adult.test', header=None, skiprows=1)

In [2]:
# subsample data frame for solver stability, comment out for erm performance
if not erm_:
    data_train = data_train.sample(frac=0.2, random_state=1)
    data_train = data_train.reset_index(drop=True)

In [3]:
n_test = data_test.shape[0]
merge = pd.concat([data_train, data_test], axis=0)
y = merge[merge.columns[-1]].map(lambda x: '>50K' in x)
merge = merge.drop(merge.columns[-1], axis=1)
X = []

for col in merge.columns:
  if merge[col].dtype == 'object':
    X.append(pd.get_dummies(merge[col], prefix=col))
  # else:
  #   X.append(merge[col])
merge_df = pd.concat(X, axis=1)
X = pd.concat(X, axis=1).to_numpy()
X_train, y_train = X[:-n_test, :], y[:-n_test]
X_test, y_test = X[-n_test:, :], y[-n_test:]
# convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6512, 102) (6512,) (16281, 102) (16281,)


In [4]:
X_train.shape, y_train.sum() / y_train.shape[0], y_test.sum() / y_test.shape[0]

((6512, 102), 0.2281941031941032, 0.23622627602727106)

In [5]:
di = None
for i, j in enumerate(merge_df.columns):
    if 'Doctorate' in j:
        di = i
        
doc_idx = X_train[:, di] == 1 
non_doc_idx = X_train[:, di] == 0

doc_idx.sum(), non_doc_idx.sum()

(74, 6438)

Init Solver

In [6]:
def lpsolver(prob, verbose=False):
#   print('=== LP Solver ===')
  solvers = [cp.MOSEK, cp.ECOS_BB]
  for s in solvers:
    # print('==> Invoking {}...'.format(s))
    try:
      result = prob.solve(solver=s, verbose=verbose)
      return result
    except cp.error.SolverError as e:
      print('==> Solver Error')

#   print('==> Invoking MOSEK simplex method...')
  try:
    result = prob.solve(solver=cp.MOSEK,
                      mosek_params={'MSK_IPAR_OPTIMIZER': mosek.optimizertype.free_simplex},
                      bfs=True, verbose=verbose)
    return result
  except cp.error.SolverError as e:
    print('==> Solver Error')

  raise cp.error.SolverError('All solvers failed.')

Make Dataset

In [7]:
class MyDataset(Dataset):
  def __init__(self, X, y):
    super(MyDataset, self).__init__()
    self.X = X.astype('float32')
    self.y = y.astype('long')
    self.attr = X

  def __getitem__(self, item):
    return self.X[item], self.y[item]

  def __len__(self):
    return len(self.X)

dataset_train = MyDataset(X_train, y_train)
dataset_test_train = MyDataset(X_train, y_train)
dataset_valid = MyDataset(X_test, y_test)
dataset_test = MyDataset(X_test, y_test)

Make Model

In [8]:
model = nn.Sequential(nn.Linear(102, 2, bias=True))
loss = nn.CrossEntropyLoss(reduction='none')

ERM

In [9]:
def erm(model: Module, loader: DataLoader, optimizer: optim.Optimizer, criterion, device: str, iters=0):
  """Empirical Risk Minimization (ERM)"""

  model.train()
  iteri = 0
  for _, (inputs, targets) in enumerate(loader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, targets).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    iteri += 1
    if iteri == iters:
      break

def test(model: Module, loader: DataLoader, criterion, device: str):
  """Test the avg and group acc of the model"""

  model.eval()
  total_correct = 0
  total_loss = 0
  total_num = 0
  l_rec = []
  c_rec = []

  with torch.no_grad():
    for _, (inputs, targets) in enumerate(loader):
      inputs, targets = inputs.to(device), targets.to(device)
      labels = targets
      outputs = model(inputs)
      predictions = torch.argmax(outputs, dim=1)
      c = (predictions == labels)
      c_rec.append(c.detach().cpu().numpy())
      correct = c.sum().item()
      l = criterion(outputs, labels).view(-1)
      l_rec.append(l.detach().cpu().numpy())
      loss = l.sum().item()
      total_correct += correct
      total_loss += loss
      total_num += len(inputs)
  l_vec = np.concatenate(l_rec)
  c_vec = np.concatenate(c_rec)
  return total_correct / total_num, total_loss / total_num, c_vec, l_vec

In [10]:
if erm_:
    batch_size = 128
    doc_idx_test = X_test[:, di] == 1
    non_doc_idx_test = X_test[:, di] == 0

    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
    trainloader = DataLoader(dataset_train, batch_size=128, shuffle=True)
    testloader = DataLoader(dataset_test, batch_size=128, shuffle=False)
    doc_testloader = DataLoader(dataset_test, batch_size=batch_size, sampler=SubsetRandomSampler(np.where(doc_idx_test)[0]), num_workers=0, pin_memory=False)
    non_doc_testloader = DataLoader(dataset_test, batch_size=batch_size, sampler=SubsetRandomSampler(np.where(non_doc_idx_test)[0]), num_workers=0, pin_memory=False) 

    erm(model, trainloader, optimizer, loss, 'cpu', iters=1000)
    acc, _, c_vec, _ = test(model, testloader, loss, 'cpu')
    print("Test Loss: ", 1 - test(model, testloader, loss, 'cpu')[0])
    print("Doc Test Loss: ", 1 - test(model, doc_testloader, loss, 'cpu')[0])
    print("Non-Doc Test Loss: ", 1 - test(model, non_doc_testloader, loss, 'cpu')[0])

    num_classes = len(np.unique(y_test))
    dct = {}
    for i in range(num_classes):
        idx = np.where(y_test == i)[0]
        dct[i] = 1 - c_vec[idx].mean()
    print("Class Loss: ", dct)
    print("Worst Class Loss: ", max(dct.values()))

    assert False

RAI GAME

In [11]:
def raigame(P, l_all, eta, constraints_w=['group_doro', 'chi2'], alpha=0.2, group_idx=[non_doc_idx]):
    num_epochs, n = l_all.shape
    w = cp.Variable(n)
    
    objective = cp.Maximize(P @ (l_all @ w) + eta * cp.sum(cp.entr(w)))
    constraints = []
    constraints.append(cp.sum(w) == 1)
    constraints.append(1e-10 <= w)
    # add constraints
    nc = len(constraints_w)
    if 'cvar' in constraints_w:
      m = alpha * n
      constraints.append(w <= 1 / m)
      nc -= 1
    if 'chi2' in constraints_w:
      m = alpha * n
      constraints.append(cp.sum_squares(w) <= 1 / m)
      nc -= 1
    if 'group_doro' in constraints_w:
      for idx in group_idx:
        constraints.append(cp.sum_squares(w[idx]) * idx.sum() <= 0.80)
      nc -= 1
    if nc > 0:
      print('Constraint {} not implemented'.format(constraints_w))
      raise NotImplementedError
    #################
    prob = cp.Problem(objective, constraints)
    result = lpsolver(prob, verbose=False)
    
    w_star = w.value
    w_star[w_star < 0] = 0
    w_star /= w_star.sum()
    return w_star, result

Train

In [12]:
init_state_dict = deepcopy(model.state_dict())
T = 15
iters_per_epoch = 500
batch_size = 128
w_all = np.zeros((0, len(dataset_train)))
l_all = np.zeros((0, len(dataset_train)))
P = np.zeros((1, 0))
n = len(dataset_train)
eta =  0.05 * (np.log(T) / (2 * T * np.log(n))) ** 0.5
print(eta, eta * np.log(n))

w_all = np.concatenate([w_all, np.ones((1, n)) / n])

test_trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False) # for testing
prev_gamevalue = np.inf
best_gamevalue, best_P = np.inf, None
models = []
for t in range(T):
    #set-up model
    model.load_state_dict(init_state_dict)
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)

    #set-up loader
    w_t = w_all[-1, :]
    sampler = WeightedRandomSampler(w_t, iters_per_epoch * batch_size, replacement=True)
    trainloader = DataLoader(dataset_train, batch_size=batch_size, sampler=sampler, num_workers=0, pin_memory=False)
    
    #get ht (or lt)    
    erm(model, trainloader, optimizer, loss, 'cpu', iters_per_epoch)
    models.append(deepcopy(model.state_dict()))
    _, _, c_t, ce_t = test(model, test_trainloader, loss, 'cpu')
    l_t = 1 - c_t # get 0-1 loss
    l_all = np.concatenate([l_all, l_t.reshape(1, -1)], axis=0)
    print('t={}, loss={}'.format(t, l_t.mean()))
    # do line search for a    
    base_a = 1 / (t + 1)
    best_value = np.inf
    best_a = 1 / (t + 1)
    for m_a in [0.1, 1, 1.5]:
        a = base_a * m_a
        if a >= 1:
            continue
        P_temp = np.concatenate([(1 - a) * P, a * np.ones((1, 1))], axis=1)
        P_temp = P_temp / P_temp.sum()
        _, value = raigame(P_temp, l_all, 0)
        if not value < np.inf:
            continue   
        if value < best_value:
            best_value = value
            best_a = a
    P = np.concatenate([(1 - best_a) * P, best_a * np.ones((1, 1))], axis=1)
    P = P / P.sum()
    
    # get new game value
    _, gamevalue = raigame(P, l_all, 0)
    print('t = {}, gamevalue = {}, max_w_star = {}'.format(t + 1, gamevalue, w_t.max()))
    
    # update best gamevalue
    if gamevalue < best_gamevalue:
        best_gamevalue = gamevalue
        best_P = P
        
    # update eta if gamevalue increases
    if prev_gamevalue < gamevalue:
        eta = eta * 2
    prev_gamevalue = gamevalue
    
    #get wt
    w_t, L_P = raigame(P, l_all, eta)
    w_all = np.concatenate([w_all, w_t.reshape(1, -1)], axis=0)

0.005069391183956586 0.04451636141358459
t=0, loss=0.17613636363636365
t = 1, gamevalue = 0.37560221476052613, max_w_star = 0.00015356265356265356
t=1, loss=0.18181818181818182
t = 2, gamevalue = 0.3535192324457604, max_w_star = 0.005075837585677506
t=2, loss=0.17536855036855037
t = 3, gamevalue = 0.3486805830701639, max_w_star = 0.00553651135725408
t=3, loss=0.17367936117936117
t = 4, gamevalue = 0.3479659828464426, max_w_star = 0.005652895424075107
t=4, loss=0.17045454545454544
t = 5, gamevalue = 0.3469469346268142, max_w_star = 0.0056470912164671245
t=5, loss=0.17137592137592136
t = 6, gamevalue = 0.3469985694557651, max_w_star = 0.00566535963389098
t=6, loss=0.17168304668304668
t = 7, gamevalue = 0.34700916441520663, max_w_star = 0.005697744600997216
t=7, loss=0.17291154791154792
t = 8, gamevalue = 0.34704582246038773, max_w_star = 0.005766462305578386
t=8, loss=0.17352579852579852
t = 9, gamevalue = 0.3470980220230002, max_w_star = 0.005907241364253679
t=9, loss=0.1712223587223587

In [13]:
best_P = np.concatenate([best_P, np.zeros((1, T - best_P.shape[1]))], axis=1)
P = best_P

In [14]:
P

array([[0.175, 0.175, 0.175, 0.175, 0.3  , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

Analysis

In [15]:
P_erm = np.zeros((1, T))
P_erm[:, 0] = 1

P_unif = np.ones((1, T)) / T

_, game_erm = raigame(P_erm, l_all, 0)
_, game_opt = raigame(P, l_all, 0)
_, game_unif = raigame(P_unif, l_all, 0)

print('game_erm = {}, game_opt = {}, game_unif = {}'.format(game_erm, game_opt, game_unif))

game_erm = 0.37560221476052635, game_opt = 0.3469469346268142, game_unif = 0.3516071116972247


In [16]:
# Analyze CVaR Loss
def get_cvar_loss(P, l, alpha):
    exp_loss = (P @ l).reshape(-1)
    exp_loss = np.sort(exp_loss)[::-1]
    n = exp_loss.shape[0]
    return format(exp_loss[:int(alpha * n)].mean(), '.3f')

for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    print('alpha = {}, opt_cvar = {}, erm_cvar = {}, unif_cvar = {}'.format(alpha, get_cvar_loss(P, l_all, alpha), get_cvar_loss(P_erm, l_all, alpha), get_cvar_loss(P_unif, l_all, alpha)))

alpha = 0.1, opt_cvar = 1.000, erm_cvar = 1.000, unif_cvar = 1.000
alpha = 0.2, opt_cvar = 0.831, erm_cvar = 0.881, unif_cvar = 0.844
alpha = 0.3, opt_cvar = 0.583, erm_cvar = 0.587, unif_cvar = 0.581
alpha = 0.4, opt_cvar = 0.437, erm_cvar = 0.440, unif_cvar = 0.436
alpha = 0.5, opt_cvar = 0.350, erm_cvar = 0.352, unif_cvar = 0.348
alpha = 0.6, opt_cvar = 0.291, erm_cvar = 0.294, unif_cvar = 0.290
alpha = 0.7, opt_cvar = 0.250, erm_cvar = 0.252, unif_cvar = 0.249
alpha = 0.8, opt_cvar = 0.219, erm_cvar = 0.220, unif_cvar = 0.218
alpha = 0.9, opt_cvar = 0.194, erm_cvar = 0.196, unif_cvar = 0.194
alpha = 1, opt_cvar = 0.175, erm_cvar = 0.176, unif_cvar = 0.174


In [17]:
# Analyze worst-class loss
def get_worst_class_loss(P, l, dataset):
    assert len(dataset) == l.shape[1]
    exp_loss = (P @ l).reshape(-1)
    num_classes = len(np.unique(dataset.y))
    dct = {}
    for i in range(num_classes):
        idx = np.where(dataset.y == i)[0]
        print(i, idx.shape)
        dct[i] = exp_loss[idx].mean()
    print("Class Loss: ", dct)
    print("Worst Class Loss: ", max(dct.values()))
    return 

get_worst_class_loss(P, l_all, dataset_train)
get_worst_class_loss(P_erm, l_all, dataset_train)

0 (5026,)
1 (1486,)
Class Loss:  {0: 0.07661659371269398, 1: 0.507150067294751}
Worst Class Loss:  0.507150067294751
0 (5026,)
1 (1486,)
Class Loss:  {0: 0.05113410266613609, 1: 0.5989232839838493}
Worst Class Loss:  0.5989232839838493


In [18]:
testloader = DataLoader(dataset_test, batch_size=128, shuffle=False)
acc = test(model, testloader, loss, 'cpu')[0]
1 - acc

0.17228671457527178

In [19]:
doc_idx_test = X_test[:, di] == 1
non_doc_idx_test = X_test[:, di] == 0

def get_acc(P, models, loader):
    acc = []
    for mod in models:
        model.load_state_dict(mod)
        acc.append(test(model, loader, loss, 'cpu')[0])
    return np.array(acc), P @ acc

from torch.utils.data.sampler import SubsetRandomSampler
doc_testloader = DataLoader(dataset_test, batch_size=batch_size, sampler=SubsetRandomSampler(np.where(doc_idx_test)[0]), num_workers=0, pin_memory=False)
non_doc_testloader = DataLoader(dataset_test, batch_size=batch_size, sampler=SubsetRandomSampler(np.where(non_doc_idx_test)[0]), num_workers=0, pin_memory=False) 

acc_doc, acc_doc_avg = get_acc(P, models, doc_testloader)
acc_non_doc, acc_non_doc_avg = get_acc(P, models, non_doc_testloader)
1 - acc_doc, 1 - acc_doc_avg, 1 - acc_non_doc, 1 - acc_non_doc_avg

(array([0.3038674 , 0.25966851, 0.24309392, 0.25414365, 0.26519337,
        0.25414365, 0.25966851, 0.25414365, 0.24861878, 0.25414365,
        0.25414365, 0.28176796, 0.28729282, 0.28176796, 0.28176796]),
 array([0.26519337]),
 array([0.1736646 , 0.17732919, 0.1773913 , 0.17062112, 0.16925466,
        0.17      , 0.16819876, 0.17      , 0.16857143, 0.17161491,
        0.17378882, 0.17465839, 0.1778882 , 0.17770186, 0.1710559 ]),
 array([0.17310248]))

In [20]:
# acc_doc, acc_doc_avg = get_acc(P_unif, models, doc_testloader)
# acc_non_doc, acc_non_doc_avg = get_acc(P_unif, models, non_doc_testloader)
# acc_doc, acc_doc_avg, acc_non_doc, acc_non_doc_avg